In [1]:
# Importation des librairies
from webscraping.ws_fonctions import *


In [5]:
notes_presse_vs_spect = get_presse_vs_spect(2021, 2, 'animation')
notes_presse_vs_spect.head(6)

,Auteurs,Note,genre
0,Presse,"4,2",animation
1,Spectateurs,"4,3",animation
2,Presse,"3,3",animation
3,Spectateurs,"3,8",animation
4,Presse,"3,7",animation
5,Spectateurs,"3,7",animation
